Install Dependencies

In [ ]:
%pip install tensorflow opencv-python matplotlib

In [ ]:
import tensorflow as tf
import os
import cv2
import imghdr
import numpy as np
from matplotlib import pyplot as plt
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Conv2D, Flatten, MaxPooling2D
from tensorflow.keras.metrics import Precision, Recall, BinaryAccuracy

In [ ]:
cpus = tf.config.experimental.list_physical_devices('CPU')
cpus

In [ ]:
#create a variable to hold the data path
data_dir = 'data'

In [ ]:
#setting the image extension to a list
image_exts = ["jpg", "jpeg", "png", "bmp"]


In [ ]:
for root,dirs, files in os.walk(data_dir):
    for file in files:
        image_path = os.path.join(root, file)
        try:
            img = cv2.imread(image_path)
            tip = imghdr.what(image_path)
            if tip not in image_exts:
                print('Image not in ext list {}'.format(image_path))
                os.remove(image_path)
        except Exception as e:
            print('Issue with image {}'.format(image_path))
            
            #os.remove(image_path)
            

Loading the data

In [ ]:
data = tf.keras.utils.image_dataset_from_directory('data')
data_iterator = data.as_numpy_iterator()

In [ ]:
#this will return a batch of images and labels. so you can rerun this cell to get another batch
batch = data_iterator.next()
# the images are represented as numpy arrays
batch[0].shape

In [ ]:
#the labels 0 or 1 represent the class of the image(happy or sad)
batch[1]

In [ ]:
#this would help us classify the labels and we can see here that 0 is happy and 1 is sad

fig, ax = plt.subplots(ncols=4, figsize=(20, 20))
for idx, img in enumerate(batch[0][:4]):
    ax[idx].imshow(img.astype("int"))
    ax[idx].title.set_text(batch[1][idx])


Data Preprocessing 

Scale the data

In [ ]:
data = data.map(lambda x, y: (x /255, y))

In [ ]:
scaled_iterator = data.as_numpy_iterator()

In [ ]:
batch = scaled_iterator.next()

In [ ]:

fig, ax = plt.subplots(ncols=4, figsize=(20, 20))
for idx, img in enumerate(batch[0][:4]):
    ax[idx].imshow(img)
    ax[idx].title.set_text(batch[1][idx])


Split the data into train and test sets

In [ ]:
train_size = int(len(data) * 0.7)
validation_size = int(len(data) * 0.2)
test_size = int(len(data) * 0.2)

In [ ]:
train = data.take(train_size)
validation = data.skip(train_size).take(validation_size)
test = data.skip(train_size + validation_size).take(test_size)

Deep learning

Building the deep learning model

In [ ]:
model = Sequential()

In [ ]:
#Adding the different layers

model.add(Conv2D(16, (3,3), 1,padding='same', activation='relu', input_shape=(256, 256, 3)))
model.add(MaxPooling2D())

model.add(Conv2D(32, (3,3), 1,padding='same', activation='relu'))
model.add(MaxPooling2D())

model.add(Conv2D(64, (3,3), 1,padding='same', activation='relu'))
model.add(MaxPooling2D())

model.add(Flatten())

model.add(Dense(256, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
#compiling the model
model.compile(optimizer='adam', loss=tf.losses.BinaryCrossentropy(), metrics=['accuracy'])

In [ ]:
model.summary()

Train the deep learning model

In [ ]:
logdir = 'logs'
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)

In [ ]:
hist = model.fit(train, epochs=20, validation_data=validation, callbacks=[tensorboard_callback])

Plot performance

In [ ]:
fig = plt.figure()
plt.plot(hist.history['loss'], color='teal', label = 'loss')
plt.plot(hist.history['val_loss'], color='orange', label = 'val_loss')
fig.suptitle('Loss', fontsize=20)
plt.legend(loc='upper right')
plt.show()

Evaluating Performance 

In [ ]:
precision = Precision()
recall = Recall()
binary_accuracy = BinaryAccuracy()

In [ ]:
for batch in test.as_numpy_iterator():
    X, y = batch
    yhat = model.predict(X)
    precision.update_state(y, yhat)
    recall.update_state(y, yhat)
    binary_accuracy.update_state(y, yhat)
    
print('Precision: ', precision.result().numpy(),
      'Recall: ', recall.result().numpy(),
      'Accuracy: ', binary_accuracy.result().numpy(),
      )

Test

In [ ]:
img = cv2.imread('sad people.jpeg')
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
plt.show()

Note: before passing the image to our neural network it must be 256 pixels high, 256 pixels wide and it need to be  3 channels

In [ ]:
resize = tf.image.resize(img, (256, 256))
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
plt.show()

In [ ]:
resize.shape

In [ ]:
np.expand_dims(resize, 0).shape

In [ ]:
yhat = model.predict(np.expand_dims(resize/255, 0))

In [ ]:
yhat

In [ ]:
if yhat < 0.5:
    print('Happy image')
else:
    print('Sad image')

Saving the model

In [ ]:
model.save(os.path.join('models', 'image_classification.h5'))